# UBER - Ride Acceptance Rates Across Geographic Zones

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_zone = pd.read_csv('Data/015/fct_zone_daily_rides.csv')
df_zone.head()

,zone_name,ride_date,total_requests,accepted_requests,declined_requests,acceptance_rate
0,Downtown,2024-04-10,100,80,20,0.800000
1,Downtown,2024-05-01,90,40,50,0.444444
2,Downtown,2024-06-20,100,90,10,0.900000
3,Suburban,2024-04-05,80,60,20,0.750000
4,Suburban,2024-05-10,80,30,50,0.375000


# Pregunta 1

### Para cada zona geográfica, ¿cuál es la tasa de aceptación mínima observada durante el segundo trimestre (Quarter 2) de 2024? Esta información ayudará a evaluar el peor desempeño de aceptación de los conductores por zona.

In [4]:
df_Q2 = df_zone[
    (df_zone['ride_date'].between('2024-04-01','2024-06-30')) 
]

respuesta1 = df_Q2.groupby('zone_name', as_index=False)['acceptance_rate'].min()

respuesta1

,zone_name,acceptance_rate
0,Airport,0.294118
1,Downtown,0.444444
2,Midtown,0.375000
3,Suburban,0.375000
4,Uptown,0.375000


```SQL 
SELECT
    zone_name,
    MIN(acceptance_rate) AS min_acceptance_rate
FROM fct_zone_daily_rides
WHERE ride_date BETWEEN '2024-04-01' AND '2024-06-30'
GROUP BY zone_name;
```

# Pregunta 2

### Haz una lista de las distintas zonas geográficas que tuvieron al menos un día en el segundo trimestre (Q2) de 2024 con una tasa de aceptación inferior al 50%. Esta lista se utilizará para identificar las zonas donde los conductores suelen ser más reacios a aceptar viajes.

```SQL

In [5]:
zonas_criticas = df_Q2[df_Q2['acceptance_rate'] <= 0.5]

respuesta2 = zonas_criticas['zone_name'].unique()

respuesta2

array(['Downtown', 'Suburban', 'Airport', 'Midtown', 'Uptown'],
      dtype=object)

```SQL
SELECT DISTINCT zone_name
FROM fct_zone_daily_rides
WHERE ride_date BETWEEN '2024-04-01' AND '2024-06-30'
  AND acceptance_rate < 0.5;
  ```

# Pregunta 3

### ¿Qué zona geográfica tuvo la tasa de aceptación de viajes más baja en un solo día durante el segundo trimestre (Q2) de 2024, teniendo además al menos 10 solicitudes de viaje rechazadas ese mismo día? Recuerda que cada fila de la tabla representa los datos de un solo día en una sola región geográfica.

### Esto nos ayuda a identificar combinaciones específicas de zona-día donde la aceptación fue especialmente deficiente, para orientar mejoras focalizadas.

In [8]:
df_filtro = df_zone[
    (df_zone['ride_date'].between('2024-04-01','2024-06-30')) &
    (df_zone['declined_requests'] >= 10)
]

resultado = df_filtro.nsmallest(1,'acceptance_rate')

resultado[['zone_name','ride_date','acceptance_rate']]

,zone_name,ride_date,acceptance_rate
8,Airport,2024-06-05,0.294118


```SQL
SELECT
    zone_name,
    ride_date,
    acceptance_rate
FROM fct_zone_daily_rides
WHERE ride_date BETWEEN '2024-04-01' AND '2024-06-30'
  AND declined_requests >= 10
ORDER BY acceptance_rate ASC
LIMIT 1;
'''